In [1]:
import pandas as pd
import terality as te
from codetiming import Timer


In [4]:
file_path = "C:\\\\Users\\\\bhask\\\\Google Drive\\\\datasets\\\\Chicago_taxi_trips\\\\"
timer = Timer(name="Terality Tests")

In [5]:
# Loading data into dataframes using Pandas
timer.start()

taxi_trips_pdf = pd.read_csv(file_path + "Taxi_Trips_2019_2020.csv")

timer.stop()

Elapsed time: 224.4380 seconds


224.4379803

In [6]:
# Loading data into dataframes using Pandas
timer.start()
taxi_trips_tdf = te.read_csv(file_path + "Taxi_Trips_2019_2020.csv")
timer.stop()

C:\\Users\\bhask\\Google Drive\\datasets\\Chicago_taxi_trips\\Taxi_Trips_2019_2020.csv:   0%| | 0.00/8.48G [00:02<?, ?B


Elapsed time: 95.9301 seconds


95.93005089999997

In [7]:
timer.start()
print("Row Count (Pandas): %d " % taxi_trips_pdf.shape[0])
timer.stop()

Row Count (Pandas): 20366397 
Elapsed time: 0.0018 seconds


0.0018355000001974986

In [8]:
timer.start()
print("Row Count (Pandas): %d " % taxi_trips_tdf.shape[0])
timer.stop()

Row Count (Pandas): 20366397 
Elapsed time: 1.1666 seconds


1.1666497999999592

In [9]:
columns = [
'trip_id',
'taxi_id',
'trip_start_timestamp',
'trip_end_timestamp',
'trip_seconds',
'trip_miles',
'pickup_census_tract',
'dropoff_census_tract',
'pickup_community_area',
'dropoff_community_area',
'fare',
'tips',
'tolls',
'extras',
'trip_total',
'payment_type',
'company',
'pickup_centroid_latitude',
'pickup_centroid_longitude',
'pickup_centroid_location',
'dropoff_centroid_latitude',
'dropoff_centroid_longitude',
'dropoff_centroid__location'
]

In [10]:
taxi_trips_pdf.columns = columns
taxi_trips_tdf.columns = columns

In [11]:
taxi_trips_pdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20366397 entries, 0 to 20366396
Data columns (total 23 columns):
 #   Column                      Dtype  
---  ------                      -----  
 0   trip_id                     object 
 1   taxi_id                     object 
 2   trip_start_timestamp        object 
 3   trip_end_timestamp          object 
 4   trip_seconds                float64
 5   trip_miles                  float64
 6   pickup_census_tract         float64
 7   dropoff_census_tract        float64
 8   pickup_community_area       float64
 9   dropoff_community_area      float64
 10  fare                        float64
 11  tips                        float64
 12  tolls                       float64
 13  extras                      float64
 14  trip_total                  float64
 15  payment_type                object 
 16  company                     object 
 17  pickup_centroid_latitude    float64
 18  pickup_centroid_longitude   float64
 19  pickup_centroid_loc

In [ ]:
# Loading data into dataframes using Terality
timer.start()

with te.disable_cache():
    contact_tdf = te.read_parquet(file_path + "contact.parquet")
    product_tdf = te.read_json(file_path + "product.json")
    txn_tdf = te.read_csv(file_path + 'txn.csv')

timer.stop()

..\\data\\contact.parquet: 100%|███████████████████████████████████████████████████| 6.68M/6.68M [00:12<00:00, 545kB/s]
..\\data\\product.json: 100%|██████████████████████████████████████████████████████| 79.9k/79.9k [00:00<00:00, 209kB/s]
..\\data\\txn.csv: 100%|██████████████████████████████████████████████████████████| 51.4M/51.4M [00:16<00:00, 3.06MB/s]


Elapsed time: 35.8867 seconds


35.8866762

In [ ]:
# Sum a column value across the entire dataframe with about 200k rows - using Pandas
timer.start()

contact_pdf.sum()['opportunity']

timer.stop()

Elapsed time: 247.8482 seconds


247.84816089999998

In [ ]:
# Sum a column value across the entire dataframe with about 200k rows - using Terality
timer.start()

contact_tdf.sum()['opportunity']

timer.stop()

Elapsed time: 10.5582 seconds


10.558202400000027

In [ ]:
# Sum a column value across the entire dataframe with about 1M rows - using Pandas
timer.start()

net_txn_amount = txn_pdf.sum()['sales']
print("Net Txn Amount: ", net_txn_amount)

timer.stop()

In [ ]:
# Sum a column value across the entire dataframe with about 1M rows - using Terality
timer.start()

net_txn_amount = txn_tdf.sum()['sales']
print("Net Txn Amount: ", net_txn_amount)

timer.stop()

In [ ]:
# Sum a column value grouped by another column across the entire dataframe with about 1M rows - using Pandas
timer.start()

txn_pdf.groupby('contact_id').sum()[['sales'] + ['redemptions']]

timer.stop()

Elapsed time: 0.3382 seconds


0.33816919999998163

In [ ]:
# Sum a column value grouped by another column across the entire dataframe with about 1M rows - using Terality
timer.start()

txn_tdf.groupby('contact_id').sum()[['sales'] + ['redemptions']]

timer.stop()

Elapsed time: 3.3017 seconds


3.30167449999999

In [ ]:
# Describe the dataframe columns and their types - using Pandas
timer.start()

txn_pdf.describe()

timer.stop()

Elapsed time: 0.2162 seconds


0.21618570000003956

In [ ]:
# Describe the dataframe columns and their types - using Terality
timer.start()

txn_tdf.describe()

timer.stop()

Elapsed time: 5.0994 seconds


5.0994470999999635

In [ ]:
# Count of rows - using Pandas
timer.start()

txn_pdf.index.size

timer.stop()

Elapsed time: 0.0001 seconds


5.649999997103805e-05

In [ ]:
# Count of rows - using Terality
timer.start()

txn_tdf.index.size

timer.stop()

Elapsed time: 0.7884 seconds


0.7883520999999973

In [ ]:
# Sort by ticker - using Pandas
timer.start()

txn_pdf.sort_values(by = 'product_id')

timer.stop()

Elapsed time: 2.2027 seconds


2.202657399999964

In [ ]:
# Sort by ticker - using Terality
timer.start()

txn_tdf.sort_values(by ='product_id')

timer.stop()

Elapsed time: 4.2181 seconds


4.218130400000064

In [ ]:
# Slicing - using Pandas
timer.start()

txn_pdf[113:211]

timer.stop()

Elapsed time: 0.0004 seconds


0.00039140000001225417

In [ ]:
# Slicing - using Terality
timer.start()

txn_tdf[113:211]

timer.stop()

Elapsed time: 0.8531 seconds


0.8531338000000233

In [ ]:
# Selecting a single row from an un-indexed dataframe - using Pandas
timer.start()

txn_pdf[txn_pdf['txn_id'] == 423603758]

timer.stop()

Elapsed time: 0.0036 seconds


0.0036301000000094064

In [ ]:
# Selecting a single row from an un-indexed dataframe - using Terality
timer.start()

txn_tdf[txn_tdf['txn_id'] == 423603758]

timer.stop()

Elapsed time: 2.3734 seconds


2.373365100000001

In [ ]:
#filters
timer.start()

txn_pdf[txn_pdf['product_id'].isin(['FPHAX', 'VSIAX', 'PRNHX', 'FKTFX', 'VEIEX'])]

timer.stop()

Elapsed time: 0.0993 seconds


0.09928609999997207

In [ ]:
#filters
timer.start()

txn_tdf[txn_tdf['product_id'].isin(['FPHAX', 'VSIAX', 'PRNHX', 'FKTFX', 'VEIEX'])]

timer.stop()

Elapsed time: 2.4200 seconds


2.420022200000062

In [ ]:
# Merging two dataframes - using Pandas
timer.start()

contact_pdf.merge(txn_pdf, on='contact_id')

timer.stop()

Elapsed time: 2.0853 seconds


2.08529580000004

In [ ]:
# Merging two dataframes - using Terality
timer.start()

contact_tdf.merge(txn_tdf, on='contact_id')

timer.stop()

Elapsed time: 9.0715 seconds


9.071548899999925

In [ ]:
# Merging two dataframes and grouping by a single column - using Pandas
timer.start()

contact_pdf.merge(txn_pdf, on='contact_id').groupby('customer_id').sum()[['sales'] + ['redemptions']]

timer.stop()

Elapsed time: 1.0281 seconds


1.0280862000000752

In [ ]:
# Merging two dataframes and grouping by a single column - using Terality
timer.start()

contact_tdf.merge(txn_tdf, on='contact_id').groupby('customer_id').sum()[['sales'] + ['redemptions']]

timer.stop()

Elapsed time: 5.7810 seconds


5.78103909999993

In [ ]:
# Merging three dataframes and grouping by two columns - using Pandas
timer.start()

contact_pdf.merge(txn_pdf, on='contact_id').merge(product_pdf, on='product_id').groupby(['customer_id', 'product_id']).sum()[['sales'] + ['redemptions']]

timer.stop()

Elapsed time: 6.5483 seconds


6.54834470000003

In [ ]:
# Merging three dataframes and grouping by two columns - using Terality
timer.start()

contact_tdf.merge(txn_tdf, on='contact_id').merge(product_tdf, on='product_id').groupby(['customer_id', 'product_id']).sum()[['sales'] + ['redemptions']]

timer.stop()

Elapsed time: 23.8555 seconds


23.855537199999958